<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/ResNet152%20without%20hidden%20dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 6.0 MB/s 
     |████████████████████████████████| 166 kB 45.7 MB/s 
     |████████████████████████████████| 182 kB 73.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 74.6 MB/s 
     |████████████████████████████████| 162 kB 75.4 MB/s 
     |████████████████████████████████| 162 kB 71.1 MB/s 
     |████████████████████████████████| 158 kB 75.7 MB/s 
     |████████████████████████████████| 157 kB 80.2 MB/s 
     |████████████████████████████████| 157 kB 73.2 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 39.4 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 72.5 MB/s 
     |████████████████████████████████| 156 kB 77.8 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: ht

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
resnet152 = tf.keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_resnet152=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet152,
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet152.trainable=False
model_resnet152.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet152 (Functional)      (None, 2048)              58370944  
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 58,381,189
Trainable params: 10,245
Non-trainable params: 58,370,944
_________________________________________________________________


In [13]:
model_resnet152.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet152 without hidden dense",
)

In [15]:
%%time
epochs=40
history = model_resnet152.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.5795 - accuracy: 0.3750

wandb: Adding directory to artifact (/content/wandb/run-20221105_071438-2iswamot/files/model-best)... Done. 1.7s


22/22 [==============================] - 220s 8s/step - loss: 1.5795 - accuracy: 0.3750 - val_loss: 1.4569 - val_accuracy: 0.3886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3838 - accuracy: 0.4332

wandb: Adding directory to artifact (/content/wandb/run-20221105_071438-2iswamot/files/model-best)... Done. 1.6s


22/22 [==============================] - 69s 3s/step - loss: 1.3838 - accuracy: 0.4332 - val_loss: 1.3082 - val_accuracy: 0.3943
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.3039 - accuracy: 0.4588

...

22/22 [==============================] - 71s 3s/step - loss: 1.0082 - accuracy: 0.6037 - val_loss: 1.0324 - val_accuracy: 0.6114
Epoch 34/40
22/22 [==============================] - 9s 365ms/step - loss: 1.0140 - accuracy: 0.6321 - val_loss: 1.0383 - val_accuracy: 0.5657
Epoch 35/40
22/22 [==============================] - ETA: 0s - loss: 1.0079 - accuracy: 0.6009

wandb: Adding directory to artifact (/content/wandb/run-20221105_071438-2iswamot/files/model-best)... Done. 1.6s


22/22 [==============================] - 70s 3s/step - loss: 1.0079 - accuracy: 0.6009 - val_loss: 1.0228 - val_accuracy: 0.5657
Epoch 36/40
22/22 [==============================] - 9s 366ms/step - loss: 0.9962 - accuracy: 0.6378 - val_loss: 1.0441 - val_accuracy: 0.5657
Epoch 37/40
22/22 [==============================] - 9s 369ms/step - loss: 1.0035 - accuracy: 0.6136 - val_loss: 1.0251 - val_accuracy: 0.6057
Epoch 38/40
22/22 [==============================] - 9s 374ms/step - loss: 0.9860 - accuracy: 0.6335 - val_loss: 1.0349 - val_accuracy: 0.5486
Epoch 39/40
22/22 [==============================] - 9s 373ms/step - loss: 1.0035 - accuracy: 0.6278 - val_loss: 1.0335 - val_accuracy: 0.5600
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.9931 - accuracy: 0.6165

wandb: Adding directory to artifact (/content/wandb/run-20221105_071438-2iswamot/files/model-best)... Done. 1.6s


22/22 [==============================] - 72s 3s/step - loss: 0.9931 - accuracy: 0.6165 - val_loss: 1.0145 - val_accuracy: 0.6629
CPU times: user 24min 33s, sys: 1min 23s, total: 25min 57s
Wall time: 34min 49s


In [16]:
wandb.finish()

accuracy,▁▃▃▄▄▆▆▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇█▇██▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▄▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▅▅▆▅▅▅▅▅▅▅▅▅▄▄▅▅▅▅▅▅▅▅▅▆▅▅▅▅▅▅▇▆▆▆▇▅▅█
val_loss,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
accuracy,0.61648
best_epoch,39
best_val_loss,1.01453
epoch,39
loss,0.99311
val_accuracy,0.66286


In [11]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
